In [ ]:
import pandas as pd
data= pd.read_csv('global-data-on-sustainable-energy.csv')
missing_values = data.isnull().sum()
length=len(data)
missing_percentage = (missing_values / len(data)) * 100
print(missing_values)


In [ ]:
missing_percentage

In [ ]:
data_types = data.dtypes
print(data_types)

Convert the types to the desired ones

In [ ]:
def remove_functuation(density):
    return density.replace(',','')
data['Entity']=data['Entity'].astype(str)
data['Density\\n(P/Km2)']=data['Density\\n(P/Km2)'].astype(str)
data['Density\\n(P/Km2)']=data['Density\\n(P/Km2)'].apply(remove_functuation)
data['Density\\n(P/Km2)']=data['Density\\n(P/Km2)'].astype(float)

In [ ]:
#https://stackoverflow.com/questions/18689823/pandas-dataframe-replace-nan-values-with-average-of-columns
#https://www.geeksforgeeks.org/remove-multiple-elements-from-a-list-in-python/
columns=data.columns.tolist()
feature_with_null=[column for column in columns if column not in ['Entity','Year','Primary energy consumption per capita (kWh/person)']]
data.fillna(data[feature_with_null].mean(),inplace=True)
data.dropna(inplace=True)
missing_values = data.isnull().sum()
print(missing_values)